# Miami Housing · Projeto de Análise e Modelagem

## Objetivo
Investigar os determinantes do preço de venda de imóveis residenciais em Miami (2016), estruturando um fluxo incremental que contempla: (i) descoberta e inspeção do conjunto de dados; (ii) limpeza e padronização (*tidy*); (iii) análise exploratória com gramática de gráficos; e (iv) preparação de artefatos para etapas posteriores de modelagem preditiva e inferencial.

## Equipe
- João Pedro Queiroz Viana  
- João Gabriel Faus Faraco


## Importações e configurações

In [9]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

DATA_DIR = Path("data")
print(DATA_DIR)

data


## Obtendo os dados

In [12]:
import pandas as pd


def load_housing_data(data_dir: Path) -> pd.DataFrame:
    '''Loads the Miami Housing dataset.

    Loads the Miami Housing dataset from the specified directory.

    Args:
        data_dir: The directory from which the dataset will be loaded.

    Returns:
        A pandas DataFrame containing the California Housing Prices dataset.
    '''
    csv_path = data_dir / 'miami-housing.csv'
    df = pd.read_csv(csv_path)
    return df

In [13]:
data = load_housing_data(DATA_DIR)

print(f'O dataset tem {data.shape[0]} linhas e {data.shape[1]} colunas.')
print('As colunas são:')
for column_name in data.columns:
    print(f'- "{column_name}"')

O dataset tem 13932 linhas e 17 colunas.
As colunas são:
- "LATITUDE"
- "LONGITUDE"
- "PARCELNO"
- "SALE_PRC"
- "LND_SQFOOT"
- "TOT_LVG_AREA"
- "SPEC_FEAT_VAL"
- "RAIL_DIST"
- "OCEAN_DIST"
- "WATER_DIST"
- "CNTR_DIST"
- "SUBCNTR_DI"
- "HWY_DIST"
- "age"
- "avno60plus"
- "month_sold"
- "structure_quality"


## Entendimento do Negócio

### O que os dados representam

Este dataset (arquivo `miami-housing.csv`, 13.932 linhas × 17 colunas) contém **registros de transações imobiliárias residenciais** na região de Miami (EUA), com:
- **Preço de venda** do imóvel,
- **Características físicas** do lote e da área construída,
- **Localização geográfica** (latitude/longitude),
- **Qualidade estrutural** do imóvel,
- **Proximidades e acessibilidade** (distâncias até mar/água, ferrovia, rodovias e centros urbanos),
- **Atributos contextuais** (ex.: mês da venda; indicador binário relacionado a ruído/ambiente — ver nota abaixo).

> **Objetivo típico de negócio:** apoiar análises de **precificação**, avaliação de **fatores que influenciam valor** (amenidades e acessibilidade), **segmentação** de portfólio e **modelagem preditiva** do preço de venda.

> **Observação importante:** a coluna `avno60plus` é um indicador binário (0/1). Pelo padrão de datasets imobiliários, é **provavelmente** um marcador de **exposição a ruído acima de 60 dB** (ex.: ruído aeronáutico “60+ dB”). Recomenda-se **confirmar no dicionário de dados oficial** do projeto.

---

### Descrição das Colunas

| Coluna              | Tipo       | Descrição de Negócio                                                                 | Unidade / Domínio                            | Exemplos / Faixa (min–max)                      |
|---------------------|------------|--------------------------------------------------------------------------------------|-----------------------------------------------|--------------------------------------------------|
| `LATITUDE`          | float      | Latitude geográfica do imóvel (para georreferenciamento e mapas).                   | Graus decimais                                 | 25.4343 – 25.9744                                |
| `LONGITUDE`         | float      | Longitude geográfica do imóvel.                                                     | Graus decimais                                 | −80.5422 – −80.1198                              |
| `PARCELNO`          | int        | Identificador cadastral do lote/parcela (parcel number).                            | ID numérico                                    | ~1.02e11 – 3.66e12                               |
| `SALE_PRC`          | float      | **Preço de venda** do imóvel (variável-alvo comum em modelos).                      | USD                                            | **72.000 – 2.650.000**                           |
| `LND_SQFOOT`        | int        | **Área do terreno (lote)**.                                                         | Pés quadrados                                  | 1.248 – 57.064                                   |
| `TOT_LVG_AREA`      | int        | **Área construída habitável** (living area).                                        | Pés quadrados                                  | 854 – 6.287                                      |
| `SPEC_FEAT_VAL`     | int        | Valor agregado por **características especiais** (ex.: piscina, lareira, deck).     | USD (valor estimado de amenidades)             | 0 – 175.020                                      |
| `RAIL_DIST`         | float      | Distância à **ferrovia** (proxy de acessibilidade/ruído).                           | Metros (aprox.)                                | 10,5 – 29.621,5                                  |
| `OCEAN_DIST`        | float      | Distância ao **oceano** (amenidade costeira).                                       | Metros (aprox.)                                | 236,1 – 75.744,9                                 |
| `WATER_DIST`        | float      | Distância ao **corpo d’água** mais próximo (lagos/canais, exceto oceano).          | Metros (aprox.)                                | 0 – 50.399,8                                     |
| `CNTR_DIST`         | float      | Distância ao **CBD** (centro principal da cidade).                                  | Metros (aprox.)                                | 3.825,6 – 159.976,5                              |
| `SUBCNTR_DI`        | float      | Distância ao **subcentro** urbano mais próximo.                                     | Metros (aprox.)                                | 1.462,8 – 110.553,8                              |
| `HWY_DIST`          | float      | Distância à **rodovia** mais próxima.                                               | Metros (aprox.)                                | 90,2 – 48.167,3                                  |
| `age`               | int        | **Idade do imóvel** na data da venda.                                               | Anos                                           | 0 – 96                                           |
| `avno60plus`        | int (0/1)  | **Indicador binário** relacionado ao ambiente; **provável** exposição a ruído ≥60 dB.| 0 = não exposto; 1 = exposto (confirmar)       | 0 ou 1 (média ~0,015)                            |
| `month_sold`        | int        | **Mês da venda** (sazonalidade de mercado).                                         | 1–12                                           | 1 – 12 (distribuição em todos os meses)          |
| `structure_quality` | int        | **Qualidade construtiva** em escala ordinal.                                        | Escala 1–5 (1=baixa, 5=alta)                   | 1–5 (moda ~4)                                    |

### Leituras rápidas de negócio

- **Preço de venda (`SALE_PRC`)** tende a ser explicado por: área construída, área do lote, qualidade, idade, amenidades (`SPEC_FEAT_VAL`) e **localização** (distâncias a mar/água/centros/rodovias).
- **Localização importa**: menor `OCEAN_DIST` e `WATER_DIST` (maior proximidade à água) geralmente aumentam valor; já menor `RAIL_DIST`/`HWY_DIST` pode ter efeitos ambíguos (acessibilidade vs. ruído).
- **Sazonalidade**: `month_sold` ajuda a capturar ciclos de demanda/oferta no ano.
- **Qualidade e idade**: `structure_quality` e `age` capturam condição do imóvel; imóveis mais novos e com melhor padrão costumam precificar acima da média.

# Relações a serem observadas

## Características físicas e estruturais:
1. Esta etapa busca investigar quais atributos dos imóveis explicam a variação nos preços de venda `SALE_PRC`. Serão analisadas variáveis como área construída `TOT_LVG_AREA`, área do lote `LND_SQFOOT`, qualidade estrutural `structure_quality` e idade `age`. O objetivo é verificar se há correlação positiva entre tamanho e preço, identificar possíveis retornos decrescentes em terrenos muito amplos, avaliar se maiores níveis de qualidade estão associados a valores mais altos e investigar se imóveis mais antigos tendem a apresentar menor valorização.

## Fatores geográficos e localização:
2. Nesta análise, o foco está em avaliar o impacto da localização sobre o valor dos imóveis, compreendendo como fatores geográficos influenciam o preço de venda `SALE_PRC`. Serão examinadas as distâncias em relação a pontos de interesse e fontes de ruído, como a proximidade ao oceano `OCEAN_DIST` e a corpos d’água `WATER_DIST`, que podem indicar valorização de áreas costeiras. Também será investigado o efeito de centralidade, observando se imóveis mais próximos ao centro urbano `CNTR_DIST` e a subcentros `SUBCNTR_DI` apresentam preços mais elevados. Além disso, será analisada a influência da distância a rodovias `HWY_DIST`, que pode ter efeitos ambíguos — associando-se tanto à acessibilidade quanto ao ruído e à poluição. Por fim, pretende-se explorar se a proximidade à ferrovia `RAIL_DIST` e a exposição a ruído (avno60plus) impactam negativamente a atratividade e o valor dos imóveis.

## Aspectos contextuais e sazonais:
3. objetivo desta etapa é analisar padrões sazonais e fatores contextuais que influenciam o preço de venda `SALE_PRC`. Serão avaliados o mês da venda `month_sold`, para identificar possíveis variações sazonais, e o valor das amenidades `SPEC_FEAT_VAL`, como piscinas ou lareiras, que podem agregar valor ao imóvel. Além disso, será considerada a variável `avno60plus`, que indica exposição a ruído, a fim de verificar se ambientes mais ruidosos tendem a apresentar menor valorização.

## Análise exploratória